# Task 2c

In [52]:
import torch
import os
import networkx as nx
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import normalized_mutual_info_score
from scipy.stats import describe
import ncut
from itertools import product

### Loading the graphs

Graph loading functions, could be changed to load multigraph directly from file

In [2]:
def csv_to_graph(path, threshold=0.3):
    # load in csv as dataframe
    df = pd.read_csv(path,header=None)
    
    # threshold dataframe and remove diagonal
    A = (df>threshold).astype(int) - pd.DataFrame(np.identity(df.shape[0]))
         
    # convert to graph
    G = nx.from_pandas_adjacency(A)
    
    return G

def load_graphs(folder):
    #load in all graphs in folder
    G1s, G2s = [],[]

    for i in range(1,61):
        filename = f'{folder}/p{i:03}_1.csv'
        G1s.append(csv_to_graph(filename))

        filename = f'{folder}/p{i:03}_2.csv'
        G2s.append(csv_to_graph(filename))
        
    return G1s, G2s

G1s, G2s = load_graphs("FC")

In [3]:
def graph_list_to_multigraph(graphs):
    G = nx.MultiGraph()
    for subgraph in graphs:
        G.add_nodes_from(subgraph.nodes)
        G.add_edges_from(subgraph.edges)
    return G

In [4]:
G1 = graph_list_to_multigraph(G1s)
G2 = graph_list_to_multigraph(G2s)

### Loading the embeddings

In [5]:
EMBEDDING_DIR = "embedding_pickles"
G1_PATH = "best_G1_DMGI.pkl"
G2_PATH = "best_G2_DMGI.pkl"

In [6]:
g1_model = torch.load(os.path.join(EMBEDDING_DIR, G1_PATH))
g1_embedding = g1_model['H'].squeeze()

g2_model = torch.load(os.path.join(EMBEDDING_DIR, G2_PATH))
g2_embedding = g2_model['H'].squeeze()

### Loading additional clinical data

In [7]:
clinical_csv = pd.read_csv("clinical.csv")
clinical_csv

,ID,SEX,MS_TYPE,AGE,EDSS,DATASET,DIAG_YEARS,BMI,THERAPY,outliers_V1,outliers_V2
0,p001,M,primary_progressive,40,7,2,11,19.918,Functional_electric_stimulation,0,14
1,p002,M,secondary_progressive,44,7,2,17,20.529,Motor_Program_Activating_Therapy,4,0
2,p003,M,primary_progressive,51,6,2,21,25.249,Functional_electric_stimulation,4,0
3,p004,M,relapsing_remitting,29,3,2,9,21.500,Functional_electric_stimulation,4,2
4,p005,F,secondary_progressive,60,6,2,21,21.411,Functional_electric_stimulation,0,4
5,p006,M,secondary_progressive,68,4,2,7,21.605,Motor_Program_Activating_Therapy,2,3
6,p007,F,relapsing_remitting,36,4,2,9,19.370,Functional_electric_stimulation,6,7
7,p008,F,relapsing_remitting,32,4,2,4,19.141,Functional_electric_stimulation,5,7
8,p009,F,relapsing_remitting,35,4,1,20,16.436,Motor_Program_Activating_Therapy,3,0
9,p010,M,secondary_progressive,54,6,2,12,23.148,Motor_Program_Activating_Therapy,20,6


## Validation of our NCut implementation
Validates our implementation of NCut against networkx by comparing the NCut values of random binary graph partitions.

In [9]:
# binary partitions validation on multigraphs

identical = True
graphs = [G1, G2]

n_checks = 10
precision = 1e-10

for i in range(n_checks):
    
        # np.random.seed(150896)

        print(f"Checking random partition no. {i}")
        clustering = np.random.randint(0, 2, size=len(G1.nodes))
        
        nodes1 = np.where(clustering==0)[0]
        nodes2 = np.where(clustering==1)[0]

        for graph in graphs:
            nx_result = nx.normalized_cut_size(graph, nodes1, nodes2)
            our_result = ncut.ncut_multigraph(graph, nodes1, nodes2)
            our_k_result = ncut.k_ncut_multigraph(graph, np.asarray([nodes1, nodes2], dtype=object))
            
            if abs(our_result - nx_result) > precision or abs(our_k_result - nx_result) > precision:
                identical = False
                break
        
        else:
            continue
        
        break

print(f"\nOur results were{' ' if identical else ' not '}identical to those of networkx for {n_checks} random partitions.")

Checking random partition no. 0
Checking random partition no. 1
Checking random partition no. 2
Checking random partition no. 3
Checking random partition no. 4
Checking random partition no. 5
Checking random partition no. 6
Checking random partition no. 7
Checking random partition no. 8
Checking random partition no. 9

Our results were identical to those of networkx for 10 random partitions.


In [78]:
# binary partitions validation on single graphs

identical = True
graphs = [G1s, G2s]

n_checks = 10
precision = 1e-10

for i in range(n_checks):
    
        print(f"Checking random partition no. {i}")
        clustering = np.random.randint(0, 2, size=len(G1.nodes))
        
        for graph in graphs:
            for subgraph in graph:
                nx_result = nx.normalized_cut_size(subgraph, np.where(clustering==0)[0], np.where(clustering==1)[0])
                our_single_result = ncut.ncut_single_graph(subgraph, np.where(clustering==0)[0], np.where(clustering==1)[0])
                # our_result = ncut.ncut_multigraph(subgraph, np.where(clustering==0)[0], np.where(clustering==1)[0])
                our_k_result = ncut.k_ncut_multigraph(subgraph,  [np.where(clustering==0)[0], np.where(clustering==1)[0]])

                if abs(our_result - nx_result) > precision or \
                    abs(our_k_result - nx_result) > precision or \
                    abs(our_result - nx_result) > precision:
                        
                    identical = False
                    break
        
        else:
            continue
        
        break

print(f"\nOur results were{' ' if identical else ' not '}identical to those of networkx for {n_checks} random partitions.")

Checking random partition no. 0
Checking random partition no. 1
Checking random partition no. 2
Checking random partition no. 3
Checking random partition no. 4
Checking random partition no. 5
Checking random partition no. 6
Checking random partition no. 7
Checking random partition no. 8
Checking random partition no. 9

Our results were not identical to those of networkx for 10 random partitions.


In [79]:
# TODO: can k-way n-cut be validated?

## Clustering the data

In [11]:
def ncut_by_k_partition(graph, partition):
    """Helper function to generate the right input structure for ncut.k_ncut_multigraph"""
    node_lists = np.asarray([np.where(partition == value)[0] for value in np.unique(partition)], dtype=object)
    return ncut.k_ncut_multigraph(graph, node_lists)

In [15]:
n_clusters = range(2, 20)

kmeans_ncut_results = []

for n in n_clusters:
    kmeans = KMeans(n_clusters=n, n_init=10)
    clustering1 = kmeans.fit_predict(g1_embedding)
    clustering2 = kmeans.fit_predict(g2_embedding)
    g1_ncut = ncut_by_k_partition(G1, clustering1)
    g2_ncut = ncut_by_k_partition(G2, clustering2)
    kmeans_ncut_results.append((f"KMeans n_clusters={n}", g1_ncut, g2_ncut, clustering1, clustering2))

In [16]:
for result in kmeans_ncut_results:
    print(result[:3])

('KMeans n_clusters=2', 0.4485409889722306, 0.498379984310383)
('KMeans n_clusters=3', 1.3399197916131274, 1.2944462057863382)
('KMeans n_clusters=4', 2.1568309204141634, 2.113798624371209)
('KMeans n_clusters=5', 2.934989958947483, 3.0425825546755423)
('KMeans n_clusters=6', 3.6024664572557485, 3.8110486163501536)
('KMeans n_clusters=7', 4.640316018879102, 4.738647847777905)
('KMeans n_clusters=8', 5.480110573949453, 5.366073798047487)
('KMeans n_clusters=9', 6.495399307541367, 6.205974297522442)
('KMeans n_clusters=10', 7.0939501459122996, 7.174016291385115)
('KMeans n_clusters=11', 8.050592394875874, 7.83790119641572)
('KMeans n_clusters=12', 8.985809870643214, 9.005215928507635)
('KMeans n_clusters=13', 9.891679874930933, 9.930695637384751)
('KMeans n_clusters=14', 10.826646347424079, 10.764710572247543)
('KMeans n_clusters=15', 11.840386359039993, 11.643600475465641)
('KMeans n_clusters=16', 12.500359900406346, 12.770812818953472)
('KMeans n_clusters=17', 13.70670141404227, 13.605

In [17]:
# trying to normalize the k-way n-cut values but unfortunately they are still dependent on k
for k, result in zip(range(2,20), kmeans_ncut_results):
    print(result[0], result[1]/k, result[2]/k)

KMeans n_clusters=2 0.2242704944861153 0.2491899921551915
KMeans n_clusters=3 0.4466399305377091 0.4314820685954461
KMeans n_clusters=4 0.5392077301035408 0.5284496560928023
KMeans n_clusters=5 0.5869979917894966 0.6085165109351085
KMeans n_clusters=6 0.6004110762092915 0.6351747693916923
KMeans n_clusters=7 0.6629022884113003 0.6769496925397007
KMeans n_clusters=8 0.6850138217436816 0.6707592247559359
KMeans n_clusters=9 0.721711034171263 0.6895526997247158
KMeans n_clusters=10 0.70939501459123 0.7174016291385115
KMeans n_clusters=11 0.7318720358978067 0.712536472401429
KMeans n_clusters=12 0.7488174892202678 0.7504346607089696
KMeans n_clusters=13 0.7608984519177641 0.7638996644142116
KMeans n_clusters=14 0.7733318819588628 0.7689078980176817
KMeans n_clusters=15 0.7893590906026662 0.7762400316977094
KMeans n_clusters=16 0.7812724937753966 0.798175801184592
KMeans n_clusters=17 0.8062765537671923 0.8003521967521314
KMeans n_clusters=18 0.8179075324553126 0.8023253020504285
KMeans n_c

## Comparison of partitions

### Comparison of NCut-values
The NCut-values of the clusterings are slightly higher for the first graph.

In [18]:
kmeans_ncut_results = [list(res) for res in kmeans_ncut_results]
result_diffs = np.asarray([res[1] - res[2] for res in kmeans_ncut_results])
describe(result_diffs)

DescribeResult(nobs=18, minmax=(-0.27045291854712517, 0.28942501001892573), mean=0.03965918484325569, variance=0.02685434201294115, skewness=-0.08877684711377885, kurtosis=-0.86674060463961)

### NMI scores
Compares how similar clusterings on G1 and G2 are by calculating the NMIs between them.

In [19]:
descriptions = [res[0] for res in kmeans_ncut_results]

kmeans_nmis = [normalized_mutual_info_score(res[3], res[4]) for res in kmeans_ncut_results]
for res, nmi in zip(descriptions, kmeans_nmis):
    print(f"{res} NMI: {nmi}")

KMeans n_clusters=2 NMI: 0.6780944524398714
KMeans n_clusters=3 NMI: 0.7770279988463307
KMeans n_clusters=4 NMI: 0.646327180797945
KMeans n_clusters=5 NMI: 0.613913961669145
KMeans n_clusters=6 NMI: 0.6004459230231852
KMeans n_clusters=7 NMI: 0.5800435164933219
KMeans n_clusters=8 NMI: 0.5806522567931678
KMeans n_clusters=9 NMI: 0.5767734672748275
KMeans n_clusters=10 NMI: 0.6360986251368707
KMeans n_clusters=11 NMI: 0.6440227834261241
KMeans n_clusters=12 NMI: 0.6320583891720516
KMeans n_clusters=13 NMI: 0.6635325641451542
KMeans n_clusters=14 NMI: 0.6877544544770605
KMeans n_clusters=15 NMI: 0.6708953486375112
KMeans n_clusters=16 NMI: 0.697820327998712
KMeans n_clusters=17 NMI: 0.7194733713770838
KMeans n_clusters=18 NMI: 0.7007599223305763
KMeans n_clusters=19 NMI: 0.695711972427002


### Comparison by attributes in clinical.csv
In this section we investigate whether the attributes in clinical.csv (such as the age) have an influence on the NCut values of the partitions. To do that, we interpret the column values of clinical.csv as cluster labels, and compute the NCut valus of the graph layers (i.e. the patients) individually. Afterwards, we group the NCut values by cluster labels and compare descriptive statistics between the groups.

In [20]:
comparison_attributes = [
    ['sex', clinical_csv['SEX']],
    ['age over 50', clinical_csv['AGE'] > 50],
    ['BMI over 25', clinical_csv['BMI'] > 25],
    ['MS type', clinical_csv['MS_TYPE']],
    ['therapy', clinical_csv['THERAPY']]
]

In [21]:
# takes a while to run

individual_ncuts = []

for res in kmeans_ncut_results:
    
    G1_ncuts = []
    G1_partition = res[3]
    for patient in G1s:
        G1_ncuts.append(ncut_by_k_partition(patient, G1_partition))

    G2_ncuts = []
    G2_partition = res[4]
    for patient in G2s:
        G2_ncuts.append(ncut_by_k_partition(patient, G2_partition))
        
    individual_ncuts.append([res[0], G1_ncuts, G2_ncuts])

In [22]:
attribute_statistics = []

for part_description, G1_ncuts, G2_ncuts in individual_ncuts:
    for comp_description, labels in comparison_attributes:
        
        label_ncuts_g1 = {}
        label_ncuts_g2 = {}
        
        for label in np.unique(labels):
            indices = np.where(labels == label)[0]
            
            # note: if desired, more descriptive statistics could be saved here
            stats = describe(np.asarray(G1_ncuts)[indices])
            label_ncuts_g1[label] = {'mean': stats.mean, 'variance': stats.variance}
            
            stats = describe(np.asarray(G2_ncuts)[indices])
            label_ncuts_g2[label] = {'mean': stats.mean, 'variance': stats.variance}
            
        attribute_statistics.append({'partition_name' : part_description, 'attribute_name' : comp_description,
                                    'G1' : label_ncuts_g1, 'G2' : label_ncuts_g2})

In [23]:
for entry in attribute_statistics:
    
    print("Partition:", entry['partition_name'])
    print("Attribute:", entry['attribute_name'], "\n")
    
    for key in entry['G1'].keys():
        print(key)
        print("G1 mean ncut:", entry['G1'][key]['mean'])
        print("G2 mean ncut:", entry['G2'][key]['mean'])
        print()
        
    print("-------------------")

Partition: KMeans n_clusters=2
Attribute: sex 

F
G1 mean ncut: 0.44666108655689596
G2 mean ncut: 0.5045774425620273

M
G1 mean ncut: 0.42536134258475505
G2 mean ncut: 0.483686714324778

-------------------
Partition: KMeans n_clusters=2
Attribute: age over 50 

False
G1 mean ncut: 0.43628292896835297
G2 mean ncut: 0.5049785283882371

True
G1 mean ncut: 0.4413904421973277
G2 mean ncut: 0.4855726861178785

-------------------
Partition: KMeans n_clusters=2
Attribute: BMI over 25 

False
G1 mean ncut: 0.4428168093559048
G2 mean ncut: 0.5086785707632295

True
G1 mean ncut: 0.43103328756955167
G2 mean ncut: 0.4756533688755535

-------------------
Partition: KMeans n_clusters=2
Attribute: MS type 

primary_progressive
G1 mean ncut: 0.44234057280040023
G2 mean ncut: 0.45672633228023773

relapsing_remitting
G1 mean ncut: 0.42996563080729894
G2 mean ncut: 0.4949442755822466

secondary_progressive
G1 mean ncut: 0.45184480989019776
G2 mean ncut: 0.5112848220393547

-------------------
Partition:

Comparing the mean NCuts for n_clusters=2 between patients of different ages, it seems that patients under 51 years have a lower average ncut value than those over 50 in graph 1, but a higher average ncut value in graph 2. A higher ncut value means that the partitions are more similar to each other. Based on this result, we can speculate that ???\
**insert more analysis here**

# Task 2d

In [47]:
functions = (
    nx.conductance,
    nx.cut_size,
    nx.edge_expansion,
    nx.mixing_expansion,
)

results = pd.DataFrame()
results['graph'] = ['G1', 'G2']

# only compare k=2 at position 0
# get the two k means clusterings
clustering1, clustering2 = kmeans_ncut_results[0][-2:]

# parameter embedding 1
params1 = (
    G1,
    list(np.where(clustering1==0)[0]),
    list(np.where(clustering1==1)[0]),
)

# parameter embedding 2
params2 = (
    G2,
    list(np.where(clustering2==0)[0]),
    list(np.where(clustering2==1)[0]),
)



for func in functions:

    results[func.__name__] = [func(*params1), func(*params2)]

results

,graph,conductance,cut_size,edge_expansion,mixing_expansion
0,G1,0.371081,6842,207.333333,0.064083
1,G2,0.387704,9037,220.414634,0.086098


In [75]:
functions = (
    nx.boundary_expansion,
    nx.node_expansion,
    nx.volume,
)


results = []

for i, n in enumerate(n_clusters):
    df = pd.DataFrame(columns=['graph', 'partition', *[f.__name__ for f in functions]])
    for (G_str, G), cluster in  product({'G1': G1, 'G2': G2}.items(), list(range(n))):
        # a bit hacky
        if G_str == 'G1': clustering = kmeans_ncut_results[i][-2]
        else: clustering = kmeans_ncut_results[i][-1]

        df.loc[len(df.index)] = [
            G_str, cluster,
            *[f(G, np.where(clustering==cluster)[0]) for f in functions]
        ]
    results.append(df)

    

In [79]:
results[2]

,graph,partition,boundary_expansion,node_expansion,volume
0,G1,0,3.461538,4.461538,15698
1,G1,1,0.757576,1.757576,74935
2,G1,2,4.272727,5.272727,15587
3,G1,3,48.500000,49.500000,548
4,G2,0,2.222222,3.222222,34461
5,G2,1,3.142857,4.142857,15928
6,G2,2,2.052632,3.052632,46400
7,G2,3,7.285714,8.285714,8173
